In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from scipy.stats import rankdata
import os
import datetime
import pyfolio as pf

In [2]:
############ Index Data Download #############

ticker2="^BSESN"
start2=datetime.date(2000, 5, 20)
end2=datetime.date(2022, 3, 31)
#data1=yf.download(tickers=ticker2,start=start2,end=end2, parse_dates=['Date'], index_col='Date')
indexdf=yf.download(tickers=ticker2, start=start2, end=end2)
del indexdf['Volume']
del indexdf['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [3]:
############ Resampling Function ##################

def aggregate(daily_df, frequency):
    daily_df.reset_index(inplace=True)
    daily_df['days'] = 1
    weekly_df = daily_df.resample(frequency, on='Date').agg({'Open':'first','High':'max', 'Low':'min','Close':'last'})
    return weekly_df

In [4]:
############ Weekly  Data Processing #############

weekly_index_df = aggregate(indexdf, 'W-Fri')
weekly_index_df['Moving_avg']=weekly_index_df['Close'].rolling(window=40).mean()
weekly_index_df.dropna(inplace=True)
weekly_index_df['Regime']=np.where((weekly_index_df['Close'] > weekly_index_df['Moving_avg']),"YES","NO")
weekly_index_df.reset_index(inplace=True)

In [5]:
######### Ticker and ISIN Merge

TickerFile=pd.read_excel("ticker2.xlsx")
unique_ticker = [str(i) for i in list(TickerFile['Ticker'].unique())]
unique_ticker = pd.DataFrame(unique_ticker,columns=['Ticker'])

IsinFile=pd.read_excel("ISIN_final.xlsx")

df=pd.merge(unique_ticker, IsinFile[["Ticker","Symbol"]], on="Ticker")

In [6]:
######### Creating Symbol List to fetch data from folder #######
SymbolLst =df["Symbol"].tolist()
SymbolLst=[tick+'.NS' for tick in SymbolLst]

In [7]:
StockDataDict = dict()
path = "Data_weekly_800stocks"
for symbol in SymbolLst:
    try:
        Stockdf = pd.read_csv(os.path.join(path,symbol+'.csv'))   
        
        Stockdf['Date']=pd.to_datetime(Stockdf['Date'], format = "%Y-%m-%d")
        Stockdf=pd.merge(Stockdf, weekly_index_df[["Date","Regime"]], on="Date")

        Stockdf['Highest_high']=Stockdf["High"].rolling(window=20).max()
        Stockdf['High_30']=Stockdf['Highest_high']+0.3*Stockdf['High']
        Stockdf['Prev_High_30']=Stockdf['High_30'].shift(1)
        Stockdf.dropna(inplace=True)
        Stockdf.reset_index(drop=True,inplace=True)
        Stockdf = round(Stockdf,2)
        StockDataDict[symbol] = Stockdf
        
    except Exception as e:
        print(symbol,e)
        
        continue

SHANTIGEAR.NS [Errno 2] No such file or directory: 'Data_weekly_800stocks\\SHANTIGEAR.NS.csv'
GDL.NS [Errno 2] No such file or directory: 'Data_weekly_800stocks\\GDL.NS.csv'


In [8]:
#df_comp_01 = StockDataDict['POLYMED.NS']

In [9]:
######## Main Loop ##########
Tradebook = pd.DataFrame(columns=['BuyDate','Symbol','BuyPrice','Regime','Stoploss','SellDate','SellPrice','PnL'])
OpenPosDict = dict()

for key,df_comp_01 in StockDataDict.items():
    OpenPosition = {'Date':None,'BuyPrice':None,'Symbol':None,'Regime':None,'Prev_high_30':None,'Stoploss':None,
                'Position':None}
    for i in range(len(df_comp_01)):
        ### Check the conditon to take new position
        if (df_comp_01['Close'][i] > df_comp_01['Prev_High_30'][i]) and (OpenPosition['Position'] is None):
            OpenPosition['Date'] =df_comp_01['Date'][i]  
            OpenPosition['BuyPrice'] =df_comp_01['Close'][i] 
            OpenPosition['Symbol'] =df_comp_01['Symbol'][i]
            OpenPosition['Regime'] =df_comp_01['Regime'][i]
            OpenPosition['Prev_high_30'] =df_comp_01['Prev_High_30'][i]
            OpenPosition['Position'] = "Long"

            if OpenPosition['Regime'] == 'YES':
                OpenPosition['Stoploss'] = OpenPosition['BuyPrice'] * 0.8
            else:
                OpenPosition['Stoploss'] = OpenPosition['BuyPrice'] * 0.9

        #### Positions is already there
        elif OpenPosition['Position'] == "Long":     
            ### Check for Stop trigger
            if df_comp_01['Close'][i] < OpenPosition['Stoploss']:
                # Stop Trigger
                SellDate = df_comp_01['Date'][i]
                SellPrice = df_comp_01['Close'][i]
                PnL = round(((SellPrice - OpenPosition['BuyPrice']) / OpenPosition['BuyPrice']),2)
                # Record the trade
                Tradebook.loc[len(Tradebook)] = [OpenPosition['Date'],OpenPosition['Symbol'],OpenPosition['BuyPrice'],
                                                 OpenPosition['Regime'],OpenPosition['Stoploss'],SellDate,SellPrice,PnL]
                # Reset the OpenPosition Dict
                OpenPosition = OpenPosition.fromkeys(OpenPosition, None)

            ## Check to Update the stoploss. Because the idea of breakout strat is to trail stoploss
            else:
                NewClose = df_comp_01['Close'][i]
                if OpenPosition['Regime'] == 'YES':
                    RevisedPotentialStop = NewClose * .8
                else:
                    RevisedPotentialStop = NewClose * .9


                if RevisedPotentialStop > OpenPosition['Stoploss']:
                    OpenPosition['Stoploss'] = RevisedPotentialStop
    OpenPosDict[key] = OpenPosition             

In [10]:
Tradebook

,BuyDate,Symbol,BuyPrice,Regime,Stoploss,SellDate,SellPrice,PnL
0,2017-03-03,MRO-TEK.NS,60.50,YES,48.400,2017-05-12,48.20,-0.20
1,2018-06-22,MRO-TEK.NS,57.10,YES,45.680,2018-06-29,44.25,-0.23
2,2021-08-20,MRO-TEK.NS,51.40,YES,56.800,2021-10-29,53.35,0.04
3,2012-01-27,INDIANCARD.NS,99.65,YES,93.912,2012-05-25,93.80,-0.06
4,2007-12-14,GILLETTE.NS,1097.86,YES,960.432,2008-01-11,925.41,-0.16
...,...,...,...,...,...,...,...,...
527,2020-08-07,AARTIDRUGS.NS,585.74,YES,783.016,2020-10-23,746.01,0.27
528,2004-12-17,BHARATRAS.NS,65.39,YES,52.312,2005-01-07,51.85,-0.21
529,2011-05-06,BHARATRAS.NS,150.18,NO,176.346,2011-05-20,169.25,0.13
530,2021-04-30,ALKYLAMINE.NS,3383.16,YES,3509.160,2021-11-05,3376.55,-0.00


In [11]:
Tradebook = Tradebook.sort_values(by ='BuyDate',ascending =True)

In [12]:
Tradebook

,BuyDate,Symbol,BuyPrice,Regime,Stoploss,SellDate,SellPrice,PnL
155,2001-08-17,HINDOILEXP.NS,17.00,NO,15.300,2001-08-24,14.12,-0.17
80,2001-10-12,DRREDDY.NS,807.43,NO,726.687,2001-10-19,427.46,-0.47
127,2002-02-15,ENGINERSIN.NS,8.22,YES,8.608,2002-03-08,8.48,0.03
241,2002-03-15,MFSL.NS,21.98,YES,20.680,2002-05-31,19.77,-0.10
30,2002-06-14,CENTURYTEX.NS,54.52,YES,49.696,2002-07-26,45.62,-0.16
...,...,...,...,...,...,...,...,...
40,2021-12-10,HCC.NS,17.40,YES,15.200,2022-03-04,15.15,-0.13
314,2021-12-10,63MOONS.NS,178.50,YES,270.400,2022-02-11,268.30,0.50
131,2021-12-31,SMARTLINK.NS,230.75,YES,184.600,2022-01-21,176.40,-0.24
417,2022-01-07,ARSSINFRA.NS,48.70,YES,38.960,2022-01-21,37.80,-0.22


In [13]:
Tradebook.reset_index(drop = True,inplace=True)

In [14]:
Tradebook

,BuyDate,Symbol,BuyPrice,Regime,Stoploss,SellDate,SellPrice,PnL
0,2001-08-17,HINDOILEXP.NS,17.00,NO,15.300,2001-08-24,14.12,-0.17
1,2001-10-12,DRREDDY.NS,807.43,NO,726.687,2001-10-19,427.46,-0.47
2,2002-02-15,ENGINERSIN.NS,8.22,YES,8.608,2002-03-08,8.48,0.03
3,2002-03-15,MFSL.NS,21.98,YES,20.680,2002-05-31,19.77,-0.10
4,2002-06-14,CENTURYTEX.NS,54.52,YES,49.696,2002-07-26,45.62,-0.16
...,...,...,...,...,...,...,...,...
527,2021-12-10,HCC.NS,17.40,YES,15.200,2022-03-04,15.15,-0.13
528,2021-12-10,63MOONS.NS,178.50,YES,270.400,2022-02-11,268.30,0.50
529,2021-12-31,SMARTLINK.NS,230.75,YES,184.600,2022-01-21,176.40,-0.24
530,2022-01-07,ARSSINFRA.NS,48.70,YES,38.960,2022-01-21,37.80,-0.22


In [15]:
Tradebook.to_excel('TradeBook.xlsx',index=None)